# Indice. Cosas hechas

- Preprocesamiento previo
    - Creación de PK
    - Quitamos datos estáticos (ETA <2400): ERROR
    - Nueva columna `predict_arrival_date`: check
    - Nueva columna `reliable_arrival_date`: FALTA COMPROBAR

- Matriz de correlaciones: casi check but peta

    
- Para cada variable: 
    - Valores nulos. Posibles soluciones
    - Correlaciones con ETA
    - Gráfico según categoría
    - Conclusión. La quitamos o la dejamos




In [1]:
import os
import polars as pl
import pandas as pd
import seaborn as sns
import polars.selectors as cs
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

import warnings

warnings.filterwarnings("ignore")

In [2]:
ROOT_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))
DATA_PATH = os.path.join(ROOT_PATH, "data", "processed")
EMT_DATA_PATH = os.path.join(DATA_PATH, "emt")


# Leemos los datos 

In [3]:
lf = pl.scan_csv(os.path.join(EMT_DATA_PATH, "2024", "03", "emt_202403.csv"))
lf.head().collect()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive
str,str,i64,str,i64,f64,f64,i64,i64,str,i64,str,str,i64,i64,bool,str,str,i64
"""2024-03-02""","""2024-03-02 07:…",579,"""27""",33,-3.702641,40.40528,0,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",1409
"""2024-03-02""","""2024-03-02 07:…",581,"""27""",33,-3.689498,40.437068,0,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",476
"""2024-03-02""","""2024-03-02 07:…",4855,"""5""",33,-3.695955,40.435,0,10663,"""CHAMARTIN""",0,null,null,null,null,false,null,null,2481
"""2024-03-02""","""2024-03-02 07:…",4857,"""5""",33,-3.688575,40.430257,0,4550,"""CHAMARTIN""",0,null,null,null,null,false,null,null,967
"""2024-03-02""","""2024-03-02 07:…",5631,"""147""",33,-3.705854,40.428887,0,4754,"""BARRIO DEL PIL…",0,null,null,null,null,false,null,null,899


# Variables numéricas

In [4]:
lf.select(cs.numeric()).head().collect()

bus,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,deviation,MinimunFrequency,MaximumFrequency,estimateArrive
i64,i64,f64,f64,i64,i64,i64,i64,i64,i64
579,33,-3.702641,40.40528,0,7528,0,9,16,1409
581,33,-3.689498,40.437068,0,2771,0,9,16,476
4855,33,-3.695955,40.435,0,10663,0,null,null,2481
4857,33,-3.688575,40.430257,0,4550,0,null,null,967
5631,33,-3.705854,40.428887,0,4754,0,null,null,899


# Variables categóricas

In [5]:
lf.select(cs.string()).head().collect()

date,datetime,line,destination,StartTime,StopTime,dayType,strike
str,str,str,str,str,str,str,str
"""2024-03-02""","""2024-03-02 07:…","""27""","""PLAZA CASTILLA…","""05:55""","""23:30""","""SA""","""N"""
"""2024-03-02""","""2024-03-02 07:…","""27""","""PLAZA CASTILLA…","""05:55""","""23:30""","""SA""","""N"""
"""2024-03-02""","""2024-03-02 07:…","""5""","""CHAMARTIN""",null,null,null,null
"""2024-03-02""","""2024-03-02 07:…","""5""","""CHAMARTIN""",null,null,null,null
"""2024-03-02""","""2024-03-02 07:…","""147""","""BARRIO DEL PIL…",null,null,null,null


# Preprocesamiento previo 
## 1. Creamos la PK

In [6]:
lf = lf.with_columns((pl.col('datetime').cast(pl.String)+"_B"+pl.col('bus').cast(pl.String)+"_L"+ pl.col('line').cast(pl.String)+"_S"+pl.col('stop').cast(pl.String)).alias('PK'))
lf.head().collect()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive,PK
str,str,i64,str,i64,f64,f64,i64,i64,str,i64,str,str,i64,i64,bool,str,str,i64,str
"""2024-03-02""","""2024-03-02 07:…",579,"""27""",33,-3.702641,40.40528,0,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",1409,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",581,"""27""",33,-3.689498,40.437068,0,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",476,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",4855,"""5""",33,-3.695955,40.435,0,10663,"""CHAMARTIN""",0,null,null,null,null,false,null,null,2481,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",4857,"""5""",33,-3.688575,40.430257,0,4550,"""CHAMARTIN""",0,null,null,null,null,false,null,null,967,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…",5631,"""147""",33,-3.705854,40.428887,0,4754,"""BARRIO DEL PIL…",0,null,null,null,null,false,null,null,899,"""2024-03-02 07:…"


## 2. Pasamos las variables `bus` y `stop` a categóricas

In [7]:
lf = lf.with_columns(pl.col('bus').cast(pl.String),pl.col('line').cast(pl.String))
lf.head().collect()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive,PK
str,str,str,str,i64,f64,f64,i64,i64,str,i64,str,str,i64,i64,bool,str,str,i64,str
"""2024-03-02""","""2024-03-02 07:…","""579""","""27""",33,-3.702641,40.40528,0,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",1409,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""581""","""27""",33,-3.689498,40.437068,0,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,false,"""SA""","""N""",476,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""4855""","""5""",33,-3.695955,40.435,0,10663,"""CHAMARTIN""",0,null,null,null,null,false,null,null,2481,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""4857""","""5""",33,-3.688575,40.430257,0,4550,"""CHAMARTIN""",0,null,null,null,null,false,null,null,967,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""5631""","""147""",33,-3.705854,40.428887,0,4754,"""BARRIO DEL PIL…",0,null,null,null,null,false,null,null,899,"""2024-03-02 07:…"


## 3. Pasamos la variable `isHead` a numérica

In [8]:
lf = lf.with_columns(pl.col('isHead').cast(pl.UInt8))
lf.head().collect()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive,PK
str,str,str,str,i64,f64,f64,i64,i64,str,i64,str,str,i64,i64,u8,str,str,i64,str
"""2024-03-02""","""2024-03-02 07:…","""579""","""27""",33,-3.702641,40.40528,0,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,0,"""SA""","""N""",1409,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""581""","""27""",33,-3.689498,40.437068,0,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,0,"""SA""","""N""",476,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""4855""","""5""",33,-3.695955,40.435,0,10663,"""CHAMARTIN""",0,null,null,null,null,0,null,null,2481,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""4857""","""5""",33,-3.688575,40.430257,0,4550,"""CHAMARTIN""",0,null,null,null,null,0,null,null,967,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""5631""","""147""",33,-3.705854,40.428887,0,4754,"""BARRIO DEL PIL…",0,null,null,null,null,0,null,null,899,"""2024-03-02 07:…"


## 4. Eliminamos los valores incorrectos `estimateArrive` 

In [9]:
lf.group_by(pl.col('estimateArrive')).count().sort('estimateArrive',descending=True).head().collect()

estimateArrive,count
i64,u32
999999,348687
888888,13029
5400,1
5399,2
5398,3


Como da error y en el TFG habla de que quita los tiempos mayores a 40 minutos, vamos a filtar por los datos menores a 2400 segundos.

In [ ]:
lf = lf.filter(pl.col('estimateArrive')<2400)

## 5. Creamos una variable `predict_arrival_date` con la predicción de la fecha de llegada

In [10]:
def calculate_remaining_time(date_str,second):
    date_datetime = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S.%f')
    new_date_datetime = date_datetime + timedelta(seconds=second)
    new_date_str = new_date_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
    
    return new_date_str
    

In [11]:
lf = lf.with_columns(pl.struct(datetime = pl.col('datetime'), estimateArrive = pl.col('estimateArrive').alias('struct')).map_elements(lambda x: calculate_remaining_time(x['datetime'], x['estimateArrive'])).alias('predict_arrival_date'))

In [12]:
lf.head().collect()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive,PK,predict_arrival_date
str,str,str,str,i64,f64,f64,i64,i64,str,i64,str,str,i64,i64,u8,str,str,i64,str,str
"""2024-03-02""","""2024-03-02 07:…","""579""","""27""",33,-3.702641,40.40528,0,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,0,"""SA""","""N""",1409,"""2024-03-02 07:…","""2024-03-02 08:…"
"""2024-03-02""","""2024-03-02 07:…","""581""","""27""",33,-3.689498,40.437068,0,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,0,"""SA""","""N""",476,"""2024-03-02 07:…","""2024-03-02 08:…"
"""2024-03-02""","""2024-03-02 07:…","""4855""","""5""",33,-3.695955,40.435,0,10663,"""CHAMARTIN""",0,null,null,null,null,0,null,null,2481,"""2024-03-02 07:…","""2024-03-02 08:…"
"""2024-03-02""","""2024-03-02 07:…","""4857""","""5""",33,-3.688575,40.430257,0,4550,"""CHAMARTIN""",0,null,null,null,null,0,null,null,967,"""2024-03-02 07:…","""2024-03-02 08:…"
"""2024-03-02""","""2024-03-02 07:…","""5631""","""147""",33,-3.705854,40.428887,0,4754,"""BARRIO DEL PIL…",0,null,null,null,null,0,null,null,899,"""2024-03-02 07:…","""2024-03-02 08:…"


**Para quitar los valores estáticos, agrupamos sobre `date`,`bus`,`line`,`stop`,`destination`,`remaining_time`**

In [ ]:
lf.group_by(pl.col('date'),pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('predict_arrival_date')).count().head().collect()

# 6. Creamos una variable `reliable_arrival_date` con la **fecha fiable** de llegada

In [9]:
lf_2_march = lf.filter(pl.col('date') == "2024-03-02")

In [11]:
lf_2_march.filter(pl.col('estimateArrive')<2400).head().collect()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive,PK
str,str,str,str,i64,f64,f64,i64,i64,str,i64,str,str,i64,i64,u8,str,str,i64,str
"""2024-03-02""","""2024-03-02 07:…","""579""","""27""",33,-3.702641,40.40528,0,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,0,"""SA""","""N""",1409,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""581""","""27""",33,-3.689498,40.437068,0,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,0,"""SA""","""N""",476,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""4857""","""5""",33,-3.688575,40.430257,0,4550,"""CHAMARTIN""",0,null,null,null,null,0,null,null,967,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""5631""","""147""",33,-3.705854,40.428887,0,4754,"""BARRIO DEL PIL…",0,null,null,null,null,0,null,null,899,"""2024-03-02 07:…"
"""2024-03-02""","""2024-03-02 07:…","""5634""","""147""",33,-3.698311,40.431891,0,8468,"""BARRIO DEL PIL…",0,null,null,null,null,0,null,null,1743,"""2024-03-02 07:…"


In [ ]:
def get_interval_time(date_str):
    date_datetime = int(datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S.%f').hour)
    
    return [date_datetime - 1, date_datetime + 1]

In [ ]:
def create_exact_time_col(date_day):
    lf_aux = lf.filter(pl.col('date') ==date_day)
    lf_aux = lf_aux.with_columns(pl.col('datetime').apply(get_interval_time).alias('interval_time'))
    
    print(lf_aux.collect().shape)
    
    small_lf = lf_aux.filter(pl.col('estimateArrive')<=60).group_by(pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('date'),pl.col('interval_time')).min().with_columns(pl.col('predict_arrival_date').alias('reliable_arrival_date'))
    
    final_lf = lf_aux.join(small_lf,on=[pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('date'),pl.col('interval_time')],how = 'left')
    
    final_lf = final_lf.select(
    pl.col('PK'),pl.col('date'),pl.col('datetime'),pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('positionBusLon'),pl.col('positionBusLat'),pl.col('positionTypeBus'),pl.col('DistanceBus'),
    pl.col('destination'),pl.col('deviation'),pl.col('StartTime'), pl.col('StopTime'),pl.col('MinimunFrequency'),pl.col('MaximumFrequency'),pl.col('isHead'),pl.col('dayType'),pl.col('strike'),pl.col('estimateArrive'),pl.col('remaining_time'),pl.col('interval_time'),pl.col('exact_time')).sort(by=[pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('date')])
    
    final_lf = final_lf.filter(pl.col('exact_time').is_not_null())
    
    return final_lf.collect()
    
    

In [ ]:
lf_aux = lf.filter(pl.col('date') =="2024-03-02")
lf_aux = lf_aux.with_columns(pl.col('datetime').apply(get_interval_time).alias('interval_time'))



In [ ]:
lf_aux.head().collect()

date,datetime,bus,line,stop,positionBusLon,positionBusLat,positionTypeBus,DistanceBus,destination,deviation,StartTime,StopTime,MinimunFrequency,MaximumFrequency,isHead,dayType,strike,estimateArrive,PK,remaining_time,interval_time
str,str,str,str,i64,f64,f64,i64,i64,str,i64,str,str,i64,i64,u8,str,str,i64,str,str,list[i64]
"""2024-03-02""","""2024-03-02 07:…","""579""","""27""",33,-3.702641,40.40528,0,7528,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,0,"""SA""","""N""",1409,"""2024-03-02 07:…","""2024-03-02 08:…","[6, 8]"
"""2024-03-02""","""2024-03-02 07:…","""581""","""27""",33,-3.689498,40.437068,0,2771,"""PLAZA CASTILLA…",0,"""05:55""","""23:30""",9,16,0,"""SA""","""N""",476,"""2024-03-02 07:…","""2024-03-02 08:…","[6, 8]"
"""2024-03-02""","""2024-03-02 07:…","""4855""","""5""",33,-3.695955,40.435,0,10663,"""CHAMARTIN""",0,null,null,null,null,0,null,null,2481,"""2024-03-02 07:…","""2024-03-02 08:…","[6, 8]"
"""2024-03-02""","""2024-03-02 07:…","""4857""","""5""",33,-3.688575,40.430257,0,4550,"""CHAMARTIN""",0,null,null,null,null,0,null,null,967,"""2024-03-02 07:…","""2024-03-02 08:…","[6, 8]"
"""2024-03-02""","""2024-03-02 07:…","""5631""","""147""",33,-3.705854,40.428887,0,4754,"""BARRIO DEL PIL…",0,null,null,null,null,0,null,null,899,"""2024-03-02 07:…","""2024-03-02 08:…","[6, 8]"


In [ ]:
print(lf_aux.collect().shape)

small_lf = lf_aux.filter(pl.col('estimateArrive')<=60).group_by(pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('date'),pl.col('interval_time')).min().with_columns(pl.col('remaining_time').alias('exact_time'))

final_lf = lf_aux.join(small_lf,on=[pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('date'),pl.col('interval_time')],how = 'left')

final_lf = final_lf.select(
pl.col('PK'),pl.col('date'),pl.col('datetime'),pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('positionBusLon'),pl.col('positionBusLat'),pl.col('positionTypeBus'),pl.col('DistanceBus'),
pl.col('destination'),pl.col('deviation'),pl.col('StartTime'), pl.col('StopTime'),pl.col('MinimunFrequency'),pl.col('MaximumFrequency'),pl.col('isHead'),pl.col('dayType'),pl.col('strike'),pl.col('estimateArrive'),pl.col('remaining_time'),pl.col('interval_time'),pl.col('exact_time')).sort(by=[pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('date')])

final_lf = final_lf.filter(pl.col('exact_time').is_not_null())

In [ ]:
lf_2_march = create_exact_time_col("2024-03-02")

(561371, 22)


In [ ]:
print(lf_2_march.sort('date','bus','line','stop','destination','estimateArrive','remaining_time','exact_time').select('datetime','estimateArrive','predict_arrival_date','reliable_arrival_date'))

shape: (388_507, 4)
┌─────────────────┬────────────────┬────────────────────────────┬────────────────────────────┐
│ datetime        ┆ estimateArrive ┆ remaining_time             ┆ exact_time                 │
│ ---             ┆ ---            ┆ ---                        ┆ ---                        │
│ str             ┆ i64            ┆ str                        ┆ str                        │
╞═════════════════╪════════════════╪════════════════════════════╪════════════════════════════╡
│ 2024-03-02      ┆ 0              ┆ 2024-03-02 10:35:02.443102 ┆ 2024-03-02 10:34:30.567009 │
│ 10:35:02.443102 ┆                ┆                            ┆                            │
│ 2024-03-02      ┆ 0              ┆ 2024-03-02 11:48:03.272066 ┆ 2024-03-02 11:48:03.272066 │
│ 11:48:03.272066 ┆                ┆                            ┆                            │
│ 2024-03-02      ┆ 0              ┆ 2024-03-02 12:56:03.059361 ┆ 2024-03-02 12:56:03.059361 │
│ 12:56:03.059361 ┆           

#### Mismo experimento pero con todos los dias

In [ ]:
lf_2_3_march = lf.with_columns(pl.col('datetime').apply(get_interval_time).alias('interval_time'))

In [ ]:
small_lf = lf_2_3_march.filter(pl.col('estimateArrive')<=60).group_by(pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('date'),pl.col('interval_time')).min().with_columns(pl.col('predict_arrival_date').alias('reliable_arrival_date'))
small_lf.head().collect()

In [ ]:
small_lf.sort(pl.col('date'),pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('estimateArrive'),pl.col('predict_arrival_date'),pl.col('exact_time')).head().collect()

In [ ]:
final_lf = lf_2_3_march.join(small_lf,on=[pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('date'),pl.col('interval_time')],how = 'left')
#final_lf.collect()

In [ ]:
PRUEBA2 = final_lf.select(
    pl.col('PK'),pl.col('date'),pl.col('datetime'),pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('positionBusLon'),pl.col('positionBusLat'),pl.col('positionTypeBus'),pl.col('DistanceBus'),
    pl.col('destination'),pl.col('deviation'),pl.col('StartTime'), pl.col('StopTime'),pl.col('MinimunFrequency'),pl.col('MaximumFrequency'),pl.col('isHead'),pl.col('dayType'),pl.col('strike'),pl.col('estimateArrive'),pl.col('remaining_time'),pl.col('interval_time'),pl.col('exact_time')).sort(by=[pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination'),pl.col('date')])

In [ ]:
PRUEBA2.head().collect()

In [ ]:
lf_2_3_march_2 = PRUEBA2.filter(pl.col('exact_time').is_not_null())

In [ ]:
DF = lf_2_3_march_2.collect()

In [ ]:
lf_2_3_march_2.head().collect()

In [ ]:
pandas_df = DF.to_pandas()

In [ ]:
pandas_df.to_csv('/home/mlia/proyectos/data-generation/docs/notebooks/aux/prueba.csv')

# Una vez que hemos realizado para 2 dias

In [ ]:
lf_aux = pl.scan_csv('/home/mlia/proyectos/data-generation/docs/notebooks/aux/prueba.csv')
lf_aux.head().collect()

In [ ]:
lf_aux=lf_aux.with_columns(pl.col('bus').cast(pl.String))

In [ ]:
lf_aux.sort(by=[pl.col('date'),pl.col('bus'),pl.col('line'),pl.col('stop'),pl.col('destination')]).head(10).collect()

# 6. Matriz de correlaciones

In [13]:
numeric_var = ['bus', 'line','stop',
 'positionBusLon',
 'positionBusLat',
 'DistanceBus',
 'deviation',
 'MinimunFrequency',
 'MaximumFrequency',
 'isHead',
 'estimateArrive',
 'predict_arrival_date']

In [14]:
matrix = [[0] * len(numeric_var) for _ in range(len(numeric_var))]
for i in range(0,len(numeric_var)):
    for j in range(0,len(numeric_var)):
        matrix[i][j] = lf.select(pl.corr(numeric_var[i],numeric_var[j])).collect().item()

In [ ]:
matrix_corr = pd.DataFrame(matrix, columns=numeric_var,index=numeric_var).apply(lambda col: col.round(4))

In [ ]:
sns.heatmap(matrix_corr, cmap='coolwarm',vmin=-1, vmax=1)

In [ ]:
matrix_corr.to_csv('/home/mlia/proyectos/data-generation/docs/notebooks/aux/matrix_corr_emt.csv', index=False)

# 7. Estudio por variables

In [ ]:
lf.head().describe() #TODO: si le quito el head peta el kernel

##  Variable `date`

### Valores nulos

In [ ]:
lf.group_by(pl.col('date')).count().head()

## Variable `datetime`

### Valores nulos

In [ ]:
lf.group_by(pl.col('datetime')).count().head()

## Varable `bus`

### Valores nulos

In [13]:
lf.group_by(pl.col('bus')).count().head()

bus,count
str,u32
"""5760""",59
"""2034""",4569
"""5660""",15869
"""5475""",13511
"""2193""",448
…,…
"""5748""",6323
"""2129""",41197
"""9120""",77401


### Correlación

In [ ]:
lf.select(pl.corr('bus','estimateArrive')).head().collect()

### Gráfico

In [ ]:
lf.group_by(pl.col('bus')).mean().select(pl.col('bus'),pl.col('estimateArrive')).collect().plot.bar(x='bus')

### Conclusión

In [ ]:
lf.filter(pl.col('estimateArrive')<=5400).group_by(pl.col('bus')).mean().select(pl.col('bus'),pl.col('estimateArrive')).sort('estimateArrive',descending=True).collect()

Según que autobús sea, el tiempo medio de espera varia bastante. Por lo que esta variable va a ser necesaria a la hora de la creación de nuestro modelo.

## Variable `line`

### Valores nulos

In [ ]:
lf.group_by(pl.col('line')).count().head()

### Correlación

In [ ]:
lf.select(pl.corr('line','estimateArrive')).head().collect()

### Gráfico

In [ ]:
lf.group_by(pl.col('line')).mean().select(pl.col('line'),pl.col('estimateArrive')).collect().plot.bar(x='line')

### Conclusión

In [ ]:
lf.group_by(pl.col('line')).mean().select(pl.col('line'),pl.col('estimateArrive')).sort('estimateArrive',descending=True).collect()

Según que línea sea, el tiempo medio de espera varia bastante. Por lo que esta variable va a ser necesaria a la hora de la creación de nuestro modelo.

## Variable `stop`

### Valores nulos

In [ ]:
lf.group_by(pl.col('stop')).count().head()

### Correlación

In [ ]:
lf.select(pl.corr('stop','estimateArrive')).head().collect()

### Gráfico

In [ ]:
lf.group_by(pl.col('stop')).mean().select(pl.col('stop'),pl.col('estimateArrive')).collect().plot.bar(x='stop')

### Conclusión

In [ ]:
lf.group_by(pl.col('stop')).mean().select(pl.col('stop'),pl.col('estimateArrive')).sort('estimateArrive',descending=True).collect()

Según que parada sea, el tiempo medio de espera varia bastante. Por lo que esta variable va a ser necesaria a la hora de la creación de nuestro modelo.

## Variable `positionBusLon`

### Valores nulos

In [ ]:
lf.group_by(pl.col('positionBusLon')).count().head()

### Correlación

In [ ]:
lf.select(pl.corr('positionBusLon','estimateArrive')).head().collect()

## Variable `positionBusLat`

### Valores nulos

In [ ]:
lf.group_by(pl.col('positionBusLat')).count().head()

### Correlación

In [ ]:
lf.select(pl.corr('positionBusLat','estimateArrive')).head().collect()

## Variable `positionTypeBus`

### Valores nulos

In [ ]:
lf.group_by(pl.col('positionTypeBus')).count().head()

### Correlación

In [ ]:
lf.select(pl.corr('positionTypeBus','estimateArrive')).head().collect()

### Gráfico

In [ ]:
lf.group_by(pl.col('positionTypeBus')).mean().select(pl.col('positionTypeBus'),pl.col('estimateArrive')).head().collect().plot.bar(x='positionTypeBus')

### Conclusión
Como no tenemos información acerca del significado de esta variable, no podemos entenderla por lo que decidimos eliminarla

In [ ]:
lf = lf.drop('positionBusType')
lf.head().collect()

In [ ]:
#lf.group_by(pl.col('positionTypeBus')).mean().select(pl.col('positionTypeBus'),pl.col('estimateArrive')).sort('estimateArrive',descending=True).collect()

## Variable `DistanceBus`

### Valores nulos

In [ ]:
lf.group_by(pl.col('DistanceBus')).count().head()

### Correlación

In [ ]:
lf.select(pl.corr('DistanceBus','estimateArrive')).head().collect()

### Conclusión
Como era de esperar, es la variable que mayor correlación tiene con el `ETA`

## Variable `destination`

### Valores nulos

In [ ]:
lf.group_by(pl.col('destination')).count().head()

### Gráfico

In [ ]:
lf.group_by(pl.col('destination')).mean().select(pl.col('destination'),pl.col('estimateArrive')).collect().plot.bar(x='destination')

### Conclusión

In [ ]:
lf.group_by(pl.col('destination')).mean().select(pl.col('destination'),pl.col('estimateArrive')).sort('estimateArrive', descending=True).collect()

Según que destino sea, el tiempo medio de espera varia bastante. Por lo que esta variable va a ser necesaria a la hora de la creación de nuestro modelo.

## Variable `deviation`

### Valores nulos

In [ ]:
lf.group_by(pl.col('deviation')).count().head()

### Correlación

In [ ]:
lf.select(pl.corr('deviation','estimateArrive')).head().collect()

In [ ]:
lf.group_by(pl.col('deviation')).mean().select(pl.col('deviation'),pl.col('estimateArrive')).sort('deviation', descending=True).collect()

No tenemos información acerca del significado de esta variable. Por lo que la eliminamos también

In [ ]:
lf = lf.drop('deviation')
lf.head().collect()

## Variable `StartTime`

### Valores nulos

In [ ]:
lf.group_by(pl.col('StartTime')).count().head()

In [ ]:
lf.group_by(pl.col('StartTime')).count().sort(pl.col('StartTime'),descending=True).collect()

## Variable `StopTime`

### Valores nulos

In [ ]:
lf.group_by(pl.col('StopTime')).count().head()

In [ ]:
lf.group_by(pl.col('StopTime')).count().sort(pl.col('StopTime'),descending=True).collect()

### Conclusion

La información que nos dan las dos variables anteriores refleja que estamos considerando solo autobuses diurnos e ignorando los nocturnos. Por tanto, esta variable no va a inferir en el tiempo de estimación ya que el hecho de que su horario comience a las 6 de la mañana o a las 7 de la mañana no va a depender de que tarde más o menos a lo largo del día. Por tanto consideramos que se pueden borrar también.

In [ ]:
lf = lf.drop('StartTime','StopTime')
lf.head().collect()

## Variable `MinimunFrequency`

### Valores nulos

In [14]:
lf.group_by(pl.col('MinimunFrequency')).count().head()

(13071377, 20)

In [15]:
#TODO: rellenar por la linea 

In [ ]:
lf.group_by(pl.col('MinimunFrequency')).count().sort(pl.col('MinimunFrequency'),descending=True).filter(pl.col('MinimunFrequency').is_not_null()).sum().collect()

### Correlación

In [ ]:
lf.select(pl.corr('MinimunFrequency','estimateArrive')).head().collect()

### Gráfico

In [ ]:
lf.group_by(pl.col('MinimunFrequency')).mean().select(pl.col('MinimunFrequency'),pl.col('estimateArrive')).collect().plot.bar(x='MinimunFrequency')

## Variable `MaximumFrequency`

### Valores nulos

In [ ]:
lf.group_by(pl.col('MaximumFrequency')).count().head()

In [ ]:
lf.group_by(pl.col('MaximumFrequency')).count().sort(pl.col('MaximumFrequency'),descending=True).collect()

### Correlación

In [ ]:
lf.select(pl.corr('MaximumFrequency','estimateArrive')).head().collect()

### Gráfico

In [ ]:
lf.group_by(pl.col('MaximumFrequency')).mean().select(pl.col('MaximumFrequency'),pl.col('estimateArrive')).collect().plot.bar(x='MaximumFrequency')

### Conclusión
Las variables `MinimunFrequency ` y `MaximumFrequency ` tienen muchos valores nulos pero si que están relacionadas con el `ETA`. Una opción es mantener solo una de ellas ya que es probable que aporten la misma información. De momento mantenemos las dos.

In [ ]:
#TODO: que hacemos con los valores nulos ??

## Variable `isHead`

### Valores nulos

In [ ]:
lf.group_by(pl.col('isHead')).count().head()

### Correlación

In [ ]:
lf.select(pl.corr('isHead','estimateArrive')).head().collect()

### Gráfico

In [ ]:
lf.group_by(pl.col('isHead')).mean().select(pl.col('isHead'),pl.col('estimateArrive')).head().collect().plot.bar(x='isHead')

### Conclusión

In [ ]:
lf.group_by(pl.col('isHead')).mean().select(pl.col('isHead'),pl.col('estimateArrive')).head().collect()

Mantenemos esta variable ya que está bastante relacionada con el `ETA`

## Variable `dayType`

### Valores nulos

In [ ]:
lf.group_by(pl.col('dayType')).count().head()

**Vemos de que tipo son los días nulos utilizando la fecha**

In [ ]:
def get_type_day(date):
    
    day = datetime.strptime(date, "%Y-%m-%d").strftime("%A")
    
    if day in ['Monday','Tuesday','Wednesday','Thursday','Friday']:
        
        type = 'LA'
    elif day == 'Saturday':
        type = 'SA'
    else:
        type = 'FE'
        
    return type

In [ ]:
lf = lf.with_columns(pl.when(pl.col('dayType').is_null()).then(pl.col('date').apply(get_type_day)).otherwise(pl.col('dayType')).alias('dayType'))
lf.head().collect()

In [ ]:
#lf.filter(pl.col('dayType').is_null()).collect().shape #TODO: peta el kernel

### Gráfico

In [ ]:
#TODO: revisar esta funcion
lf.group_by(pl.col('dayType')).mean().select(pl.col('dayType'),pl.col('estimateArrive')).head().collect().plot.bar(x='dayType')

### Conclusión

In [ ]:
lf.group_by(pl.col('dayType')).mean().select(pl.col('dayType'),pl.col('estimateArrive')).sort('estimateArrive',descending=True).collect()

Mantenemos esta variable.

## Variable `strike`

### Valores nulos

In [ ]:
lf.group_by(pl.col('strike')).count().head()

In [ ]:
lf_reemplazado.select(pl.corr('strike','estimateArrive')).head().collect()

### Gráfico

In [ ]:
lf.group_by(pl.col('strike')).mean().select(pl.col('strike'),pl.col('estimateArrive')).head().collect().plot.bar(x='strike')

### Conclusión

In [ ]:
lf.group_by(pl.col('strike')).mean().select(pl.col('strike'),pl.col('estimateArrive')).head().collect()

- La variable `Strike` toma solo 'N' o nulo, por lo que para ningún día se tiene constancia de que hubo huelga. Por tanto para estos datos esta variable no va a aportar información.

In [ ]:
lf = lf.drop('strike')
lf.head().collect()

## Variable `estimateArrive`

### Valores nulos

In [ ]:
lf.group_by(pl.col('estimateArrive')).count().head()

### Resumen


In [ ]:
lf.select(pl.col('estimateArrive')).describe()

In [ ]:
matrix_corr

### Conclusion


- Las variables `MinimunFrequency ` y `MaximumFrequency ` están muy relacionadas entre ellas. Se podría dejar tan solo `MinimumFrequency` ya que aporta mas informacion al `ETA` que la otra


 

In [ ]:
lf = lf.drop('MaximumFrequency')
lf.head().collect()

# Conclusión

Despues de nuestro análisis terminamos con un dataset con 17 variables, las cuales son:

- `PK`
- `date`
- `datetime`
- `bus`
- `line`
- `stop`
- `positionBusLon`
- `positionBusLat` 
- `DistanceBus`
- `destination`
- `MinimunFrequency`
- `isHead`
- `dayType`
- `estimateArrive`
- `remaining_time`
- `exact_time`

In [ ]:
#TODO: fecha fromato fecha 
#TODO: cambiar nombres de variables 

## DUDAS:
- Como ejecutar los comandos con los que me peta el kernel BBDD
- Matriz de correlacion al principio o al final?? PRINCIPIO
- Como quitar valores nulos de `MinimunFrequency`